In [1]:
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
import numpy as np

# Define parameters
theta = Parameter('theta')
h = Parameter('h')
Jz = Parameter('Jz')


pauli_list = ["ZI", "IZ", "XI", "IX", "ZZ"]
coeffs = [-h * theta.cos(), -h * theta.cos(), -h * theta.sin(), -h * theta.sin(), -Jz]

H = SparsePauliOp(pauli_list, coeffs)

print(H)

SparsePauliOp(['ZI', 'IZ', 'XI', 'IX', 'ZZ'],
              coeffs=[ParameterExpression(-1.0*h*cos(theta)),
 ParameterExpression(-1.0*h*cos(theta)),
 ParameterExpression(-1.0*h*sin(theta)),
 ParameterExpression(-1.0*h*sin(theta)), ParameterExpression(-1.0*Jz)])


In [2]:
print(H.parameters)

ParameterView([Parameter(Jz), Parameter(h), Parameter(theta)])


In [3]:
# Function to compute the derivative of H with respect to theta
def differentiate_H(H, theta):
    new_coeffs = [coeff.gradient(theta) for coeff in H.coeffs]

    return SparsePauliOp(H.paulis, new_coeffs)

dH_dtheta = differentiate_H(H, theta)
print(dH_dtheta)

SparsePauliOp(['ZI', 'IZ', 'XI', 'IX', 'ZZ'],
              coeffs=[ParameterExpression(1.0*h*sin(theta)),
 ParameterExpression(1.0*h*sin(theta)),
 ParameterExpression(-1.0*h*cos(theta)),
 ParameterExpression(-1.0*h*cos(theta)), 0j])


In [4]:
alpha = Parameter('alpha')

ksi_pauli_list = ['YI', "IY"]
ksi_coeffs = [1/2 * alpha, 1/2 * alpha]

ksi = SparsePauliOp(ksi_pauli_list, ksi_coeffs)

print(ksi)

SparsePauliOp(['YI', 'IY'],
              coeffs=[ParameterExpression(0.5*alpha), ParameterExpression(0.5*alpha)])


In [5]:
print(ksi.parameters)

ParameterView([Parameter(alpha)])


In [6]:
from qiskit.quantum_info import commutator

comm_ksi_H = commutator(ksi, H).simplify()

print(comm_ksi_H)

SparsePauliOp(['XI', 'ZI', 'XZ', 'IX', 'IZ', 'ZX'],
              coeffs=[ParameterExpression(-1.0*I*alpha*h*cos(theta)),
 ParameterExpression(1.0*I*alpha*h*sin(theta)),
 ParameterExpression(-1.0*I*Jz*alpha),
 ParameterExpression(-1.0*I*alpha*h*cos(theta)),
 ParameterExpression(1.0*I*alpha*h*sin(theta)),
 ParameterExpression(-1.0*I*Jz*alpha)])


In [7]:
G = dH_dtheta + 1j * comm_ksi_H
G = G.simplify()
print(G)

SparsePauliOp(['ZI', 'IZ', 'XI', 'IX', 'XZ', 'ZX'],
              coeffs=[ParameterExpression(-1.0*alpha*h*sin(theta) + 1.0*h*sin(theta)),
 ParameterExpression(-1.0*alpha*h*sin(theta) + 1.0*h*sin(theta)),
 ParameterExpression(1.0*alpha*h*cos(theta) - 1.0*h*cos(theta)),
 ParameterExpression(1.0*alpha*h*cos(theta) - 1.0*h*cos(theta)),
 ParameterExpression(1.0*Jz*alpha), ParameterExpression(1.0*Jz*alpha)])


In [8]:
G_sq = G.power(2).simplify()

print(G_sq)

SparsePauliOp(['II', 'ZZ', 'ZX', 'IX', 'XZ', 'XI', 'XX', 'IZ', 'ZI', 'YY'],
              coeffs=[ParameterExpression(2.0*Jz**2*alpha**2 + 2.0*(-1.0*alpha*h*sin(theta) + 1.0*h*sin(theta))**2 + 2.0*(1.0*alpha*h*cos(theta) - 1.0*h*cos(theta))**2),
 ParameterExpression(2.0*(-1.0*alpha*h*sin(theta) + 1.0*h*sin(theta))**2),
 ParameterExpression(2.0*(-1.0*alpha*h*sin(theta) + 1.0*h*sin(theta))*(1.0*alpha*h*cos(theta) - 1.0*h*cos(theta))),
 ParameterExpression(2.0*Jz*alpha*(-1.0*alpha*h*sin(theta) + 1.0*h*sin(theta))),
 ParameterExpression(2.0*(-1.0*alpha*h*sin(theta) + 1.0*h*sin(theta))*(1.0*alpha*h*cos(theta) - 1.0*h*cos(theta))),
 ParameterExpression(2.0*Jz*alpha*(-1.0*alpha*h*sin(theta) + 1.0*h*sin(theta))),
 ParameterExpression(2.0*(1.0*alpha*h*cos(theta) - 1.0*h*cos(theta))**2),
 ParameterExpression(2.0*Jz*alpha*(1.0*alpha*h*cos(theta) - 1.0*h*cos(theta))),
 ParameterExpression(2.0*Jz*alpha*(1.0*alpha*h*cos(theta) - 1.0*h*cos(theta))),
 ParameterExpression(2.0*Jz**2*alpha**2)])


In [9]:
def trace_sparse_pauli_op(op):
    num_qubits = op.num_qubits  
    trace_value = 0

    for pauli, coeff in zip(op.paulis, op.coeffs):
        if pauli.to_label() == "I" * num_qubits:  
            trace_value += coeff * (2 ** num_qubits)  

    return trace_value

# Compute the trace of G^2
trace_G_sq = trace_sparse_pauli_op(G_sq)

print("Trace of G^2:", trace_G_sq)



Trace of G^2: 8.0*Jz**2*alpha**2 + 8.0*(-1.0*alpha*h*sin(theta) + 1.0*h*sin(theta))**2 + 8.0*(1.0*alpha*h*cos(theta) - 1.0*h*cos(theta))**2


In [10]:
type(trace_G_sq)

qiskit.circuit.parameterexpression.ParameterExpression

In [11]:
partial_alpha = trace_G_sq.gradient(alpha)

print(partial_alpha)

16.0*Jz**2*alpha - 16.0*h*(-1.0*alpha*h*sin(theta) + 1.0*h*sin(theta))*sin(theta) + 16.0*h*(1.0*alpha*h*cos(theta) - 1.0*h*cos(theta))*cos(theta)


In [12]:
import sympy as sp

def solve_for_alpha(expr, alpha):
    sym_expr = expr.sympify()  # Convert to SymPy
    sym_alpha = sp.Symbol(str(alpha))  # Define alpha in SymPy
    solution = sp.solve(sym_expr, sym_alpha)  # Solve for alpha
    return solution

# Solve for alpha*
alpha_star = solve_for_alpha(partial_alpha, alpha)[0]
print("Optimal alpha:", alpha_star)


Optimal alpha: h**2/(Jz**2 + h**2)


In [13]:
type(alpha_star)

sympy.core.mul.Mul

In [14]:
# Convert sympy object to a Qiskit Parameter object

alpha_star_param = Parameter(str(alpha_star))

print(alpha_star_param)
print(type(alpha_star_param))

h**2/(Jz**2 + h**2)
<class 'qiskit.circuit.parameter.Parameter'>


In [15]:
ksi_evaluated = ksi.assign_parameters({alpha: alpha_star_param})

print(ksi_evaluated)


SparsePauliOp(['YI', 'IY'],
              coeffs=[ParameterExpression(0.5*h**2/(Jz**2 + h**2)),
 ParameterExpression(0.5*h**2/(Jz**2 + h**2))])
